# **Recomendador de Musica con Base de Datos de Spotify**

En esta notebook se realizará un motor de Recomendación de Canciones a partir de una base de datos pequeña de spotify.
Empezaré con un EDA para saber como utilizar el dataset. Luego ya que tenga entienda la base la ajustare al problema para que pueda ser mas facil de utilizarla.

# **Librerias**

In [ ]:
#pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

from yellowbrick.target import FeatureCorrelation

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import json

import warnings
warnings.filterwarnings("ignore")

# **Leemos los Datos**

In [ ]:
data = pd.read_csv("data/data_tracks.csv")

In [ ]:
print(data.info())

# Analisis de Datos

Vamos a comprobar todos los parametros con la columna **'popularidad'**. Antes de ir a hacer eso vamos a comprobar la Correlación de Características considerando algunas características y para eso, voy a utilizar el paquete **yellowbrick**. Puedes aprender más sobre él en la página [documentation](https://www.scikit-yb.org/en/latest/index.html).

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','explicit','key','mode','year']

X, y = data[feature_names], data['popularity']

# Creamos una lista con los nombres de las caracteristicas
features = np.array(feature_names)

# Creamos la instancia
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(20,20)
visualizer.fit(X, y)
visualizer.show()

En estadística, el coeficiente de correlación de Pearson es una medida de dependencia lineal entre dos variables aleatorias cuantitativas. A diferencia de la covarianza, la correlación de Pearson es independiente de la escala de medida de las variables.

De manera menos formal, podemos definir el coeficiente de correlación de Pearson como un índice que puede utilizarse para medir el grado de relación de dos variables siempre y cuando ambas sean cuantitativas y continuas. 


Correlación positiva: significa que si la característica A aumenta, la característica B también aumenta o si la característica A disminuye, la característica B también disminuye. Ambas características se mueven en tándem y tienen una relación lineal.


Correlación negativa: significa que si la característica A aumenta, la característica B disminuye y viceversa.

Sin correlación: No hay relación entre esos dos atributos.

## Cantidad de canciones por década

In [ ]:
def get_decade(year):
    period_start = int(year/10) * 10
    decade = '{}s'.format(period_start)
    return decade

data['decade'] = data['year'].apply(get_decade)

sns.set(rc={'figure.figsize':(12 ,8)})
sns.countplot(data['decade'])

# **Agrupación de canciones con K-Means**

StandardScaler() normalizará las características (cada columna de X, INDIVIDUALMENTE !!!) para que cada columna/característica/variable tenga mean = 0 y standard deviation = 1. 


K-means es un algoritmo de clasificación no supervisada (clusterización) que agrupa objetos en k grupos basándose en sus características. El agrupamiento se realiza minimizando la suma de distancias entre cada objeto y el centroide de su grupo o cluster. Se suele usar la distancia cuadrática.

# MLFlow

In [ ]:
import mlflow
import mlflow.sklearn

track_uri = "http://localhost:5000/" # Esto puede ser que cambie por http://0.0.0.0:1234
mlflow.set_tracking_uri(track_uri)
mlflow.set_registry_uri("sqlite:////tmp/registry.db")

In [ ]:
# Generando el experimento o cargandolo si existe
experiment_name = "Bot_Recomendador"
mlflow.set_experiment(experiment_name)

# Cargando la información
client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

# Vamos a ver si es cierto
print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.tracking.get_tracking_uri()}")
print(f"Nombre del experimento: {experiment_name}")
print(f"ID del experimento: {experiment_id}")

In [ ]:
#number_cols =['acousticness', 'danceability', 'energy', 'instrumentalness',
#            'liveness', 'speechiness', 'tempo', 'valence','duration_ms','key','year']
number_cols = ['valence', 'danceability', 'energy', 'key', 'loudness','popularity', 'speechiness', 'tempo','year']


#number_cols = ['danceability','popularity','year']
n_c = 4

mlflow.sklearn.autolog()
params = {"n_clusters": n_c}

#Generamos un flujo de trabajo con dos procesos
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(**(params), 
                                   verbose=False))
                                 ], verbose=False)

In [ ]:
#pasamos las columnas que utilizaremos para el entrenamiento
X = data[number_cols]
song_cluster_pipeline.fit(X)
#Predecimos el grupo que pertenece cada canción
song_cluster_labels = song_cluster_pipeline.predict(X)
#Asignamos los labels del resultado de nuestro entrenamiento a nuestro dataframe
data['cluster_label'] = song_cluster_labels

mlflow.end_run()

In [ ]:
mlflow.start_run()
mlflow.sklearn.autolog()

from sklearn.decomposition import PCA

# Visualizing the Clusters with PCA

#Generamos un flujo de trabajo con dos procesos
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
#Vectorizamos las caracteristicas para encontrar los componentes principales
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['id'] = data['id']
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])

if not os.path.exists("images"):
    os.mkdir("images")

fig.write_image("images/PCA.jpeg")

plt.savefig
mlflow.log_artifact("images/PCA.jpeg")

projection.to_csv('PCA_projection.csv')
mlflow.log_artifact('PCA_projection.csv')

#mlflow.end_run()

fig.show()

In [ ]:
projection.head()

In [ ]:
length = len(number_cols)
for i in range(length):
    projection[number_cols[i]] = projection.id.map(data.set_index('id')[number_cols[i]])
    
df = projection.sort_values('cluster')
for i in range(n_c):
    globals()['cluster%s' % i]=df.loc[df.loc[:, 'cluster'] == int(i)]
cluster0.head(10)

In [ ]:
cluster0.describe()

In [ ]:
cluster1.describe()

In [ ]:
cluster2.describe()

In [ ]:
cluster3.describe()

# **Construcción del Motor de recomendación**

* Según el análisis y las visualizaciones, está claro que los géneros similares tienden a tener puntos de datos que se ubican cerca unos de otros, mientras que los tipos de canciones similares también se agrupan.
* Esta observación tiene mucho sentido. Los géneros similares sonarán de manera similar y provendrán de períodos de tiempo similares, mientras que lo mismo puede decirse de las canciones dentro de esos géneros. Podemos usar esta idea para construir un sistema de recomendación tomando los puntos de datos de las canciones que un usuario ha escuchado y recomendando canciones correspondientes a puntos de datos cercanos.
* [Spotipy](https://spotipy.readthedocs.io/en/2.16.1/) es un cliente de Python para la API web de Spotify que facilita a los desarrolladores la obtención de datos y la consulta de canciones en el catálogo de Spotify. Tienes que instalar usando `pip install spotipy`
* Después de instalar Spotipy, deberá crear una aplicación en la [página del desarrollador de Spotify] (https://developer.spotify.com/) y guardar su ID de cliente y clave secreta.

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="62cfe79d83ef43c69a7ba63f9f5debda",
                                                           client_secret="88e6707695634f04a75301fcc77d7789"))

def find_song(name):
    song_data = defaultdict()
    results = sp.search(q= 'track: {}'.format(name), limit=1)
    if results['tracks']['items'] == []:
        return None
    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['explicit'] = [int(results['explicit'])]
    # song_data['duration'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]
    # song_data['loudness'] = [results['loudness']]
    # song_data['danceability'] = [results['danceability']]
    # song_data['energy'] = [results['energy']]
    # song_data['tempo'] = [results['tempo']]
    list_of_dict_values = list(results.values())
    results_string = json.dumps(list_of_dict_values)
    index_release_date = results_string.find("release_date")
    year_index1 = index_release_date + 16
    year_index2 = index_release_date + 17
    year_index3 = index_release_date + 18
    year_index4 = index_release_date + 19
    year_string = results_string[year_index1] + results_string[year_index2] + results_string[year_index3] + results_string[year_index4]
    print(year_string)
    
    song_data['year'] = [int(year_string)]
    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)


In [ ]:
def get_song_data(song, spotify_data):
    
  #  try:
  #      song_data = spotify_data[(spotify_data['name'] == song['name'])].iloc[0]
  #      print(song['name'], ': si esta')
  #      print(song['year'])
  #      return song_data
    
  # except IndexError:
  #      print(song['name'], ': no esta')
    return find_song(song['name'])
        

def get_mean_vector(song_list, spotify_data):
    
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)


def flatten_dict_list(dict_list):
    
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict


def recommend_songs( song_list, spotify_data, n_songs=3):
    metadata_cols = ['id','name','year','artists']
    song_dict = flatten_dict_list(song_list)
    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols]

In [ ]:
recommend_songs([{'name': 'Lobo Domesticado'}],  data)

* Esta última celda te dará una lista de recomendación de canciones

* Puedes cambiar la lista de canciones según tu elección.

In [ ]:
rec=recommend_songs([{'name': 'Lobo Domesticado'}],  data)
rec.to_csv('Recomendacion.csv')
mlflow.log_artifact('Recomendacion.csv')
mlflow.end_run()